**Table of contents**<a id='toc0_'></a>    
- [移動ロボットによる強化学習を用いたライントレース](#toc1_)    
- [参考サイトなど](#toc2_)    
- [pybulletの起動](#toc3_)    
- [初期設定](#toc4_)    
- [Q学習クラスの定義](#toc5_)    
  - [Q学習とは](#toc5_1_)    
  - [Q値と報酬](#toc5_2_)    
  - [Q値の更新](#toc5_3_)    
  - [Qテーブル](#toc5_4_)    
- [Qテーブルの初期化](#toc6_)    
- [Q学習のパラメータ設定](#toc7_)    
- [Q学習の実行](#toc8_)    
- [おまけ：ライントレース用の画像生成](#toc9_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Line Tracing Using Reinforcement Learning with a Mobile Robot](#toc0_)

In this notebook, we will perform line tracing using a two-wheeled mobile robot with reinforcement learning.

(For a manual summarizing the functions available in pybullet, please refer to [here](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)


# <a id='toc3_'></a>[Starting pybullet](#toc0_)

Start pybullet.

In [1]:
import time
import math
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc4_'></a>[Initial Setup](#toc0_)

Perform initial setup such as generating the floor, creating box objects, generating the robot, and setting the camera position.


In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity to Earth's gravity
time_step = 1./240.
pybullet.setTimeStep(time_step)

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf", basePosition=[-5.0, -5.0, 0.0], globalScaling=5.0)
tex_uid = pybullet.loadTexture("../texture/line_trace_ground.png")
pybullet.changeVisualShape(plane_id, -1, textureUniqueId=tex_uid)

# Load the robot
car_start_pos = [0.0, 0.0, 0.1]  # Set initial position (x, y, z)
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set initial orientation (roll, pitch, yaw)
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf", car_start_pos, car_start_orientation)

# Set camera position for GUI mode
camera_distance = 6.0
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
camera_target_position = [0.0, 1.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


In [3]:
# Define rotation matrices to calculate cameraUpVector according to the movement of the mobile robot
def Rx(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def Ry(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def Rz(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

# <a id='toc5_'></a>[Definition of Q-Learning Class](#toc0_)

## <a id='toc5_1_'></a>[What is Q-Learning?](#toc0_)

Q-Learning is a type of reinforcement learning where the robot learns to select actions $a$ that maximize the Q-value $Q(s, a)$ when it takes a state $s$.

In this case, as shown in the figure below:

- The state $s_t$ is defined as the "color on the left" and "color on the right" obtained from the mobile robot's camera.
- The action $a_t$ is defined as the robot's driving direction (straight, turn right, turn left).

We will perform Q-Learning based on these definitions.

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/sim_environment.png" width="80%">

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/sim_environment.png" width="80%">

## <a id='toc5_2_'></a>[Q-Values and Rewards](#toc0_)

In Q-learning, the following two concepts are important:

- "Q-value $Q(s, a)$"
- "Reward $r$"

These are similar but different concepts.

"Reward $r$" represents the reward obtained after taking action $a$ in state $s$.
In this case,
- After taking action $a_t$ in state $s_t$, if the "left color is black" and "right color is black", a reward $r_{t+1}=+1$ is given,
- If the "left color is white" and "right color is white", a reward $r_{t+1}=-1$ is given,
- In other cases, no reward is given

This is set up as described above (also called immediate reward).

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/reward.png" width="100%">

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/reward.png" width="100%">

On the other hand, the "Q-value $Q(s, a)$" represents the value of taking action $a$ in state $s$, **considering future rewards**.


---

---

---

---

tips: Specific Example of the Difference Between Reward and Q-Value

Let's compare rewards and Q-values with a specific example.
Since the current example is a bit difficult to understand, let's use the example of "Go".
- In the case of Go, the reward is "win or lose". A reward of $r=+1$ is given for a win, and a reward of $r=-1$ is given for a loss.
- On the other hand, the Q-value represents "the probability of ultimately winning if the next move $a$ is made in the current situation $s$."

In 2015, DeepMind's "AlphaGo" succeeded in defeating the world champion by learning the strength of Go using Q-values.
This was the result of learning the strategy of "taking actions that maximize future rewards (victory), even if they seem disadvantageous at the moment" with high accuracy (surpassing human professional players).

Note that AlphaGo uses a method called Deep Q-Network (DQN), which combines Q-learning with neural networks.

---

---

---

---

## <a id='toc5_3_'></a>[Updating Q-Values](#toc0_)

In Q-learning, learning is performed by repeatedly updating the Q-values using the following equation:

$$
Q(s_t, a_t) = Q(s_t, a_t) + \alpha \left( r_{t+1} + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t) \right)
$$

Here, the meanings of each variable are as follows:
- $\alpha$: Learning rate
- $r_{t+1}$: Reward obtained immediately after taking action $a_t$ in state $s_t$
- $\gamma$: Discount rate (how much future rewards are considered)
- $\max_{a} Q(s_{t+1}, a)$: The highest Q-value among the possible actions $a$ in state $s_{t+1}$

<br>

The term $r_{t+1} + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t)$ is called the "TD error".  
The TD error can be defined as $\delta = r_{t+1} + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t)$, as shown below (this definition is also used in the code).

$$
Q(s_t, a_t) = Q(s_t, a_t) + \alpha \delta
$$

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/td_error.png" width="60%">

<br>
<br>

In particular, the term $r_{t+1} + \gamma \max_{a} Q(s_{t+1}, a)$ is called the "TD target", and the Q-value is updated to approach this TD target.

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/td_target.png" width="60%">

<!-- ---

---

tips:

他の強化学習手法に、SARSA（State-Action-Reward-State-Action）という手法があります。

$$
Q(s_t, a_t) = Q(s_t, a_t) + \alpha \left( r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t) \right)
$$

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/q_learn_and_sarsa.png" width="60%">

Q学習が「最もQ値が高い行動を選択する」のに対して、SARSAは「実際に取った行動に対するQ値を更新する」という違いがあります。

---

--- -->

## <a id='toc5_4_'></a>[Q-Table](#toc0_)

So far, we have explained Q-values, but since Q-values exist for each state $s$ and action $a$ pair, each Q-value is managed in a table called a "Q-Table".
 
| states \ actions | $a_0$ | $a_1$ |...| $a_n$ |
|:-----------------:|:-----:|:-----:|:-:|:-----:|
| $s_0$             | $Q(s_0, a_0)$ | $Q(s_0, a_1)$ |...| $Q(s_0, a_n)$ |
| $s_1$             | $Q(s_1, a_0)$ | $Q(s_1, a_1)$ |...| $Q(s_1, a_n)$ |
| ...               | ... | ... |...| ... |
| $s_m$             | $Q(s_m, a_0)$ | $Q(s_m, a_1)$ |...| $Q(s_m, a_n)$ |



The Q-table for this line tracing problem is shown below.

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/q_table.png" width="100%">


In this line tracing problem,
- Go straight when "black-black"
- Turn right when "white-black"
- Turn left when "black-white"

Therefore, the Q-table after learning should look like the figure below (the parts labeled "High" should have high Q-values after learning).

<img src="../images/MobileRobot/mobile_robot_reinforcement_q_learning/q_table_after_learning.png" width="60%">


*Note: For educational purposes, we have set up a simple problem where the Q-table after learning can be predicted.  
In reality, it is difficult to predict the Q-table for complex problems, so it is important to set appropriate rewards and other parameters.


In [4]:
from enum import IntEnum
import numpy as np
import cv2

class Actions(IntEnum):
    """
    Definition of actions (columns of Q-table)
    """
    STRAIGHT = 0
    TURN_LEFT = 1
    TURN_RIGHT = 2

class States(IntEnum):
    """
    Definition of states (rows of Q-table)
    """
    BLACK_BLACK = 0
    WHITE_BLACK = 1
    BLACK_WHITE = 2
    WHITE_WHITE = 3


class QLearning:
    def __init__(self, q_table, LEARNING_RATE=0.1, DISCOUNT_RATE=0.9, EPSILON=0.1):
        """
        Initialization of Q-learning

        Parameters
        ----------
        q_table : list
            Q-table
        LEARNING_RATE : float
            Learning rate
        DISCOUNT_RATE : float
            Discount rate
        EPSILON : float
            Probability of selecting a random action
        """ 
        self.q_table = q_table
        self.LEARNING_RATE = LEARNING_RATE
        self.DISCOUNT_RATE = DISCOUNT_RATE
        self.EPSILON = EPSILON   

    def get_state(self, car_id, CAMERA_IDX, CAMERA_TARGET_IDX, projection_matrix):
        """
        Get state

        Parameters
        ----------
        car_id : int
            ID of the robot
        CAMERA_IDX : int
            Index of the camera link
        CAMERA_TARGET_IDX : int
            Index of the virtual link for the target point
        projection_matrix : list
            Projection matrix

        Returns
        -------
        state : int
            State s_t
        """
        # Default direction of camera_up_vector
        camera_up_vector = np.array([0, -1, 0])
        
        # Get the position of the camera link
        camera_link_pose = pybullet.getLinkState(car_id, CAMERA_IDX)[0]

        # Get the position of the virtual link for the target point
        camera_target_link_pose = pybullet.getLinkState(car_id, CAMERA_TARGET_IDX)[0] 

        # Rotate cameraUpVector to match the posture of the mobile robot
        mobile_robot_roll, mobile_robot_pitch, mobile_robot_yaw = pybullet.getEulerFromQuaternion(pybullet.getLinkState(car_id, CAMERA_IDX)[1])
        R = Rz(np.deg2rad(90.0) + mobile_robot_yaw) @ Ry(mobile_robot_pitch) @ Rx(mobile_robot_roll)
        rotate_camera_up_vector = R @ camera_up_vector

        # Get the viewMatrix in the direction from the camera link to the virtual link for the target point
        view_matrix = pybullet.computeViewMatrix(cameraEyePosition=[camera_link_pose[0], camera_link_pose[1], camera_link_pose[2]], cameraTargetPosition=[camera_target_link_pose[0], camera_target_link_pose[1], camera_target_link_pose[2]], cameraUpVector=[rotate_camera_up_vector[0], rotate_camera_up_vector[1], rotate_camera_up_vector[2]])
        
        # Get the image of the line
        width, height, rgb_img, _, _ = pybullet.getCameraImage(600, 300, view_matrix, projection_matrix, renderer=pybullet.ER_BULLET_HARDWARE_OPENGL, shadow=0, flags=pybullet.ER_NO_SEGMENTATION_MASK)

        # Binarize the image
        img = np.reshape(rgb_img, (height, width, 4))  # Convert the obtained image to a 4-channel numpy array
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        _, binary_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)  # Binarize

        # Get the color of the pixel slightly to the left of the center of the image
        color_left = binary_img[height // 2, width // 2 - 150]

        # Get the color of the pixel slightly to the right of the center of the image
        color_right = binary_img[height // 2, width // 2 + 150]

        # Get the state
        if color_left == 0 and color_right == 0:
            state = States.BLACK_BLACK
        elif color_left == 255 and color_right == 0:
            state = States.WHITE_BLACK
        elif color_left == 0 and color_right == 255:
            state = States.BLACK_WHITE
        elif color_left == 255 and color_right == 255:
            state = States.WHITE_WHITE
        return state

    def get_action(self, state):
        """
        Get action

        Parameters
        ----------
        state : int
            State s_t

        Returns
        -------
        action : int
            Action a_t
        """

        # Action selection using ε-greedy method
        if np.random.uniform(0, 1) < self.EPSILON:
            # Select a random action with probability epsilon
            action = np.random.choice(list(Actions))
        else:
            # Select the action with the maximum Q value with probability 1-epsilon
            action = Actions(np.argmax(self.q_table[state][:]))
        return action
    
    def get_reward(self, state):
        """
        Get reward

        Parameters
        ----------
        state : int
            State s_t

        Returns
        -------
        reward : float
            Reward r_t
        """
        if state == States.BLACK_BLACK:
            reward = 1
        elif state == States.BLACK_WHITE:
            reward = 0
        elif state == States.WHITE_BLACK:
            reward = 0
        elif state == States.WHITE_WHITE:
            reward = -1
        return reward
    
    def update_q_table(self, state, action, state_next, reward):
        """
        Update Q-table

        Parameters
        ----------
        state : int
            State s_t at time t
        action : int
            Action a_t at time t
        state_next : int
            State s_{t+1} at time t+1
        reward : float
            Reward r_{t+1} at time t+1
            ※ The reward obtained as a result of transitioning from "state s{t} at time t" to "state s_{t+1} at time t+1" by taking "action a_t at time t", so the subscript is t+1
        """
        # Find the maximum Q value among the possible actions "a" in "state s_{t+1} at time t+1"
        max_q_value = max(self.q_table[state_next][:])

        # Calculate TD error
        td_error = reward + self.DISCOUNT_RATE * max_q_value - self.q_table[state][action]
                                                            
        # Update Q-table
        self.q_table[state][action] = self.q_table[state][action] + self.LEARNING_RATE * td_error
        

# <a id='toc6_'></a>[Initialization of Q-Table](#toc0_)

Initialize the Q-table.

Even if the learning process is interrupted, you can resume learning from where you left off by saving the values of this Q-table.

In [5]:
q_table = np.zeros((len(States), len(Actions)))

# <a id='toc7_'></a>[Setting Parameters for Q-Learning](#toc0_)

Set the parameters for Q-learning.

By changing the following parameters, the speed and performance of learning will vary.

In [ ]:
EPISODE_MAX = 1000 # Maximum number of episodes
STEP_MAX = 4000 # Maximum number of steps per episode
LEARNING_RATE = 0.01 # Learning rate
DISCOUNT_RATE = 0.9 # Discount rate
EPSILON = 0.1 # Epsilon for ε-greedy method (probability of selecting a random action)
LEARNING_COMPLETE_EPISODE_NUM = 4 # Number of episodes to determine learning completion (In this case, learning is considered complete if the mobile robot can complete a lap without going off the line for "LEARNING_COMPLETE_EPISODE_NUM" consecutive times)

# <a id='toc8_'></a>[Execution of Q-Learning](#toc0_)

Q-Learning is executed in the following steps:

1. Obtain the initial state $s_t$
2. Select and execute the action $a_t$ with the highest Q-value in state $s_t$ from the Q-table
    - However, a random action is selected with a certain probability (ε-greedy method)
3. Obtain the next state $s_{t+1}$
4. Obtain the reward $r_{t+1}$
5. Update the Q-value $Q(s_t, a_t)$ and reflect it in the Q-table
6. Update the state $s_t$ to $s_{t+1}$
7. Repeat steps 2 to 6 until learning is complete or the maximum number of iterations is reached

In [6]:
# Set the robot to the initial position
car_start_pos = [3.0, 0, 0.1]
car_start_orientation = pybullet.getQuaternionFromEuler([0.0, 0.0, -math.pi/2])
pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)

# Bottom camera settings
projection_matrix = pybullet.computeProjectionMatrixFOV(fov=140.0, aspect=1.0, nearVal=0.04, farVal=100)

# Link indices
CAMERA_IDX = 8
CAMERA_TARGET_IDX = 9

# Joint indices
RIGHT_WHEEL_JOINT_IDX = 0
LEFT_WHEEL_JOINT_IDX = 1

# Speed when moving straight
BASE_SPEED = 30

# Initialize Q-learning
q_learning = QLearning(q_table, LEARNING_RATE, DISCOUNT_RATE, EPSILON)
episode_complete_cnt = 0 # Number of times the mobile robot has completed a lap without going off the line consecutively

# Start learning
for episode in range(EPISODE_MAX):

    # End if learning is complete
    if episode_complete_cnt == LEARNING_COMPLETE_EPISODE_NUM:
        pybullet.removeAllUserDebugItems()
        pybullet.addUserDebugText(f"Learning is complete!!", [-2.3, 6.0, 0.1], textSize=2, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"Episode: {episode}", [-0.7, 5.5, 0.1], textSize=1.5, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"Q table", [-0.5, 5.0, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"| STATE \ ACTION | STRAIGHT | TURN LEFT | TURN RIGHT |", [-3.9, 4.6, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"----------------------------------------------------------------------------", [-3.9, 4.4, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"|   BLACK-BLACK   |    {q_table[0][0]:.2f}       |     {q_table[0][1]:.2f}        |      {q_table[0][2]:.2f}        |", [-3.9, 4.2, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"|   WHITE-BLACK   |    {q_table[1][0]:.2f}       |     {q_table[1][1]:.2f}        |      {q_table[1][2]:.2f}        |", [-3.9, 3.8, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"|   BLACK-WHITE   |    {q_table[2][0]:.2f}       |     {q_table[2][1]:.2f}        |      {q_table[2][2]:.2f}        |", [-3.9, 3.4, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        pybullet.addUserDebugText(f"|   WHITE-WHITE   |    {q_table[3][0]:.2f}       |     {q_table[3][1]:.2f}        |      {q_table[3][2]:.2f}        |", [-3.9, 3.0, 0.1], textSize=1.3, textColorRGB=[1,0,0])
        break

    # Set the mobile robot to the initial position and speed 0
    # Change the direction of travel for each episode
    if episode % 2 == 0:
        car_start_pos = [3.0, 0, 0.1]
        car_yaw = -math.pi/2
    else:
        car_yaw = math.pi/2
        car_start_pos = [3.0, 0, 0.1]
    car_start_orientation = pybullet.getQuaternionFromEuler([0.0, 0.0, car_yaw])
    pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)
    pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=0)
    pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=0)

    # Display Q table for each episode
    # (In reality, the Q table is updated every step, but addUserDebugText is heavy, so it is displayed for each episode)
    pybullet.removeAllUserDebugItems()
    pybullet.addUserDebugText(f"Episode: {episode+1}", [-0.7, 5.5, 0.1], textSize=1.5, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"Q table", [-0.5, 5.0, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"| STATE \ ACTION | STRAIGHT | TURN LEFT | TURN RIGHT |", [-3.9, 4.6, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"----------------------------------------------------------------------------", [-3.9, 4.4, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"|   BLACK-BLACK   |    {q_table[0][0]:.2f}       |     {q_table[0][1]:.2f}        |      {q_table[0][2]:.2f}        |", [-3.9, 4.2, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"|   WHITE-BLACK   |    {q_table[1][0]:.2f}       |     {q_table[1][1]:.2f}        |      {q_table[1][2]:.2f}        |", [-3.9, 3.8, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"|   BLACK-WHITE   |    {q_table[2][0]:.2f}       |     {q_table[2][1]:.2f}        |      {q_table[2][2]:.2f}        |", [-3.9, 3.4, 0.1], textSize=1.3, textColorRGB=[1,0,1])
    pybullet.addUserDebugText(f"|   WHITE-WHITE   |    {q_table[3][0]:.2f}       |     {q_table[3][1]:.2f}        |      {q_table[3][2]:.2f}        |", [-3.9, 3.0, 0.1], textSize=1.3, textColorRGB=[1,0,1])

    # 1. Get the initial state
    state = q_learning.get_state(car_id, CAMERA_IDX, CAMERA_TARGET_IDX, projection_matrix)

    # Take a maximum of STEP_MAX actions in one episode
    for step in range(STEP_MAX):

        # 2. Select and execute an action from the Q table
        action = q_learning.get_action(state)
        if action == Actions.STRAIGHT:
            left_speed = BASE_SPEED
            right_speed = BASE_SPEED
        elif action == Actions.TURN_LEFT:
            left_speed = BASE_SPEED - 10
            right_speed = BASE_SPEED
        elif action == Actions.TURN_RIGHT:
            left_speed = BASE_SPEED
            right_speed = BASE_SPEED - 10
        pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_speed)
        pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_speed)

        # Advance the simulation by one time step
        pybullet.stepSimulation()
        time.sleep(1./240.)

        # 3. Get the next state
        state_next = q_learning.get_state(car_id, CAMERA_IDX, CAMERA_TARGET_IDX, projection_matrix)

        # 4. Get the reward
        reward = q_learning.get_reward(state_next)

        # 5. Update the Q table
        q_learning.update_q_table(state, action, state_next, reward)

        # 6. Update the state
        state = state_next

        # End the episode if the mobile robot goes off the line
        if state_next == States.WHITE_WHITE:
            episode_complete_cnt = 0
            break

        # End the episode if the mobile robot completes a lap (returns near the starting point)
        current_car_pos = pybullet.getBasePositionAndOrientation(car_id)[0]
        current_to_start_distance = (car_start_pos[0] - current_car_pos[0])**2 + (car_start_pos[1] - current_car_pos[1])**2
        if current_to_start_distance < 0.1**2 and step > 1000:
            episode_complete_cnt += 1
            break


KeyboardInterrupt: 

: 

# <a id='toc9_'></a>[Bonus: Generating an Image for Line Tracing](#toc0_)

The following code generates an image with a circle of the specified radius.

In [9]:
from PIL import Image, ImageDraw

dpi = 100

plane_width = 10 
plane_height = 10
line_radius = 3

image_width = (plane_width * dpi)
image_height = (plane_height * dpi)

image = Image.new('RGB', (image_width, image_height), (255, 255, 255))
draw = ImageDraw.Draw(image)

def gazebo_to_image_coords(x, y):
    x_image = int(((x + plane_width/2) / plane_width) * image_width)
    y_image = int((1 - (y + plane_height/2) / plane_height) * image_height)
    return x_image, y_image

center_x_gazebo, center_y_gazebo = 0, 0

center_x_image, center_y_image = gazebo_to_image_coords(center_x_gazebo, center_y_gazebo)
radius_image = int(line_radius * image_width / plane_width)

draw.ellipse((center_x_image - radius_image, center_y_image - radius_image, center_x_image + radius_image, center_y_image + radius_image), outline="black", width=15)

image.save('../texture/line_trace_ground.png', dpi=(dpi, dpi))

X connection to :0 broken (explicit kill or server shutdown).


: 